# Run Example 1

This notbook shows the workflow for running EpiSim37 simulations on your personal computer.

This notebook expects that you have installed pandas and episim37.

In [1]:
import tempfile
from pathlib import Path

import pandas as pd

from episim37.utils import OpenMPSimulator, Simulation

For running an EpiSim37 simulation we need two things:
1) the EpiSim37 simulation description (the esl file) and
2) the network (nodes and edges)

For this demo we shall use the following files.
The node and edge file can be in parquet or CSV format.

In [2]:
sim_file = "example1.esl37"
node_file = "nodes.parquet"
edge_file = "edges.parquet"

The node table contains the node key (pid for this simulation) and any other static node attributes that the simulation may depend on.

Similarly, the edge table contains the source and target node key (source_pid and target_pid for this simulation) and any other static edge attributes that the simulation may depend on.

The column names in the node and edge file must match those provided in the simulation file.

In [3]:
pd.read_parquet(node_file).head()

,pid,is_in_school
0,3723936,True
1,3723937,True
2,3723938,False
3,3723939,True
4,3723940,True


In [4]:
pd.read_parquet(edge_file).head()

,target_pid,source_pid,duration,is_school_edge,is_non_home_edge
0,3723936,3723937,14400,False,True
1,3723936,3723937,20519,False,True
2,3723936,3723937,41520,False,True
3,3723936,3731305,1020,False,True
4,3723936,3742032,1320,False,True


We shall create a working directory for EpiSim37.

EpiSim37 shall use that folder to store the generated C++ codes and the compiled simulator.

We shall also store the input.h5 and output.h5 files in that work directory.

In [5]:
work_dir = Path(Path.home() / "episim37-work-dir")

input_file = work_dir / "input.h5"

In [6]:
simulator = OpenMPSimulator(sim_file, work_dir)

Next we shall call the prepare_build() method.
This uses CMake to find or fetch the C++ libraries needed for building the simulator.
It is not necessary to re-execute this method every time the simulation file is changed.

In [7]:
simulator.prepare_build()

-- Version: 10.1.0
-- Build type: 
-- Configuring done (2.0s)
-- Generating done (0.0s)
-- Build files have been written to: /home/parantapa/episim37-work-dir/gen_code/build


The build method actually builds the simulator.
It first compules the ESL37 code to C++ and then uses a C++ compiler to build the simulator binary.
It is necessary to rerun this method every time the simulation file is updated.

In [8]:
simulator.build()

[ 50%] Built target fmt
[ 66%] Generating simulator_openmp.cpp, simulator_openmp_common.h
[ 66%] Generating simulator_openmp.cpp, simulator_openmp_common.h
[ 83%] Building CXX object CMakeFiles/simulator_openmp.dir/simulator_openmp.cpp.o
[100%] Linking CXX executable simulator_openmp
[100%] Built target simulator_openmp


Now that the simulator binary has been built, we can add the network to create a full simulation.

In [9]:
simulation = Simulation(simulator, node_file, edge_file, input_file)

The prepare input step creates the input_file from the network files (node and edge file).
This also provides other relevant metadata to the simulator.

In [10]:
simulation.prepare_input()

Reading node table.

Sorting node table.

Making node index.

### num_nodes:  83313


Reading edge table.

Sorting edge table.

Computing incoming incidence CSR graph's indptr.

### num_edges:  3269342


Creating input file.

Input file created successfully.

We can view the node and edge tables in the input file using extract_nodes() and extract_edges() helper methods.

In [11]:
simulation.extract_nodes().to_pandas().head()

,pid,is_in_school
0,3723936,1
1,3723937,1
2,3723938,0
3,3723939,1
4,3723940,1


In [12]:
simulation.extract_edges().to_pandas().head()

,target_pid,source_pid,duration,is_school_edge,is_non_home_edge,_target_node_index,_source_node_index
0,3723936,3723937,14400,0,1,0,1
1,3723936,3723937,20519,0,1,0,1
2,3723936,3723937,41520,0,1,0,1
3,3723936,3731305,1020,0,1,0,7369
4,3723936,3733825,5700,1,1,0,9889


Finally to run a simulation, we provide the location of the output file and call simulate.
Here we can also pass a dictionary of configuration variables that were defined in the simulation.

In [13]:
output_file = work_dir / "output.h5"

simulation.simulate(output_file, num_ticks=10, configs={"enable_hybrid_learning": True}, verbose=True)

### num_threads = 12
### enable_hybrid_learning = 1
### enable_day_30_antigen_test = 1
### enable_day_30_pcr_test = 0
### transmissibility_scale = 0.3
### num_ticks = 10
### input_file = /home/parantapa/episim37-work-dir/input.h5
### output_file = /home/parantapa/episim37-work-dir/output.h5
### num_nodes = 83313
### num_edges = 3269342
### cur_tick = -1
### cur_tick = 0
### cur_tick = 1
### cur_tick = 2
### cur_tick = 3
### cur_tick = 4
### cur_tick = 5
### cur_tick = 6
### cur_tick = 7
### cur_tick = 8
### cur_tick = 9
### mem_use_gb = 0.0761555
### init_time_s = 0.0982847
### main_time_s = 0.0952616


Once the simulation is complete we can extract summary, transitions, transmissions and interventions from the output file.

In [14]:
simulation.extract_summary(output_file).to_pandas().head()

,S,E,Ipresymp,Isymp,Iasymp,R,tick
0,76410,6903,0,0,0,0,-1
1,76405,6901,0,0,7,0,0
2,76400,3369,3453,0,91,0,1
3,76367,2918,3456,0,572,0,2
4,76017,2091,6,3453,1746,0,3


In [15]:
simulation.extract_transitions(output_file).to_pandas().head()

,node_index,state,tick
0,77753,Iasymp,0
1,77991,Iasymp,0
2,79823,Iasymp,0
3,80914,Iasymp,0
4,81287,Iasymp,0


In [16]:
simulation.extract_transmissions(output_file).to_pandas().head()

,edge_index,state,tick
0,136443,E,2
1,196059,E,2
2,293655,E,2
3,396110,E,2
4,659180,E,2


In [17]:
simulation.extract_interventions(output_file).to_pandas().head()

,node_index,state,tick
0,3527,E,-1
1,8795,E,-1
2,59124,E,-1
3,66364,E,-1
4,74228,E,-1
